In [ ]:
#Imports 

import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.fftpack
from scipy.fft import fft, ifft
from scipy import stats
import math
import librosa
import IPython.display as ipd
import pandas as pd
from scipy.interpolate import interp1d
from scipy.io.wavfile import write
from scipy import interpolate
from scipy.optimize import minimize
import music21

# Constants

In [ ]:
# Constants for reading .times or .ideal files

SECONDS_PER_AT_FRAME = 256/8000

fft_size=1024


hop_size=int(fft_size/4)
sr_ensemble=44100


fft_lim =513 #largest bin size for cutoff

#X seconds/sample * 512 samples/STFT frame

def seconds_to_stft_frames(seconds):
    samples = seconds * sr_ensemble
    return  math.floor((samples)/hop_size)


def stft_frames_to_seconds(stft_frames):
    samples = fft_size + (stft_frames-1)*hop_size
    return samples/sr_ensemble
    
    


half_step = 1/12 #multiply this by change to get 

# User specified Values

Single best volume setting for each instrument. Order is same as instrument list

In [ ]:
nonvar_mix = np.array([[0.1222869 ],
       [0.26452896* .75],
       [0.20447949*2],
       [0.63013774],
       [0.2666939*.75 ],
       [1.8622239 ],
       [0.322688  ],
       [0.3939167* 1.2 ]], dtype=np.float32)

### Files describing original timepoints and unaltered audio



In [ ]:
#path to tuning file
path_to_ensemble_tun = '../../tuning/mozart_serenade_eflat.'

In [ ]:
path_to_ensemble_times="../../orchestra_part_times/mozart_serenade_eflat." #partial path for parsing

path_to_ensemble_audio="../../orchestra_part_audio/mozart_serenade_eflat." #partial path for parsing

instrument_lst = ["oboe_1", 
                  "oboe_2", 
                  "clarinet_1", 
                  "clarinet_2", 
                  "bassoon_1", 
                  "bassoon_2", 
                  "horn_in_e_1", 
                  "horn_in_e_2"]

#number of parts
num_voices=8
#sanity check
num_voices == len(instrument_lst)



Path to midi specification. Order of parts may be different in midi when read in by music21, if so use mid_instrument_lst variable. 

In [ ]:
path_to_mid ="../../mozart_serenade_eflat/mozart_serenade_eflat.mid"
mid_instrument_lst= ['oboe_1',
 'oboe_2',
 'clarinet_1',
 'clarinet_2',
                      'horn_in_e_1',
 'horn_in_e_2',
 'bassoon_1',
 'bassoon_2',
]

Load Original Parts Audio

In [ ]:
#Load each file into a list
ensemble_audio_lst = []
sr_dct={}
for instr in instrument_lst:
    print(instr)
    samples, sr = librosa.load(path_to_ensemble_audio+instr+".wav", sr=None)
    if sr==sr_ensemble:
        samples_48k=samples
    else:
        samples_48k = librosa.resample(samples, sr, sr_ensemble, 'kaiser_fast')
    ensemble_audio_lst.append(samples_48k)


#should be (num_instruments, length_samples)
ensemble_audio_lst

## Create list of STFTs for each part 

In [ ]:
#
part_stfts = [ ]

for i in  range(0, num_voices):
    part_stfts.append(
        librosa.stft(ensemble_audio_lst[i], n_fft=fft_size, hop_length=hop_size, center=True))


In [ ]:
part_stfts[0].shape

### Extract data from each times file and tunign file

In [ ]:
#use -1 notation on pickup
def eval_measure_fun(measure_str, quarter_per_measure = 4, pickup = False):
    mixed_lst = measure_str.split('+')
    measures = eval(mixed_lst[0])
    beats = eval(mixed_lst[1])
    measure_len = quarter_per_measure * .25 #assume quarter is .25
    
    return measures*measure_len + beats

In [ ]:



df_dct={} #list with dataframe corresponding to each one 



for instr in instrument_lst:
    

    '''
    Read in .times file as dataframe
    '''
    times_df = pd.read_csv(path_to_ensemble_times+instr+".times", 
                              header=None,
                              delim_whitespace=True,
                              names=['measure', 'AT', 'marked'])


    '''
    Read in tuning file
    '''
    tun_df = pd.read_csv(path_to_ensemble_tun+instr+".tun", 
                              header=None,
                              delim_whitespace=True,
                              names=['measure','shift (half step)'])

    #parse out the word 'solo' so can join with times df
    remove_solo = lambda x: x[5:len(x)]

    tun_df['measure']=list(map(remove_solo, tun_df['measure']))
    
    '''
    get absolute pitch shift by multiplying by half step constant 
    '''
    get_abs_tun = lambda x: 2 ** (-1*x*half_step)#Guessing positive value -> flatter
    
    tun_df['tune (abs)'] = list(map(get_abs_tun, tun_df['shift (half step)']))

    '''
    Don't care about marked
    '''
    times_df = times_df[['measure', 'AT']]

    '''
    Remove extra rows
    '''

    times_df=times_df[4:len(times_df)-1].reset_index(drop=True)



    '''
    Join
    '''
    times_df= times_df.set_index('measure').join(tun_df.set_index('measure')).reset_index()
    
    '''
    use mapping to cast measure
    '''

    evaluatedMeasure=list(map(eval_measure_fun, times_df['measure']))

    times_df['eval_measure']=evaluatedMeasure        



    '''
    use mapping to AT to float
    '''
    
    evalfun =lambda x: eval(x)
    try:
        evaluatedsec=list(map(evalfun, times_df['AT']))
        times_df['AT']=evaluatedsec
    except:
        print("casting done automatically by pandas")

    '''
    Pad last element AT (so it doesn't cut off)
    '''
    times_df['AT'].iloc[-1]= times_df.iloc[-1]['AT']+30

    '''
    Convert AT to seconds
    '''
    times_df['Seconds']=times_df['AT']*SECONDS_PER_AT_FRAME 

    '''
    Get STFT frame number at each timepoint
    '''

    evaluatedstft=list(map(seconds_to_stft_frames, times_df['Seconds']))
      

    times_df['STFT frames']=evaluatedstft

    '''
    Add to dictionary
    '''
    df_dct[instr]=times_df




# Use midi file to remove "ghost notes", i.e. additional time points not corresponding to note onsets in the .times file. 

Note this has downside of removing note offsets, if offset is not in the combined rhythm set note will just keep going

In [ ]:
'''
For a given part gets music21 notes/chords, measure metric used in Cadenza, 
    and each note's offset in quarter notes
Params:
part: music21.stream.Part, part to parse

Returns: 

list of music21 notes/chords, measure metric used in Cadenza, 
    and each note's offset in quarter notes

Returns :
offsets, eval_measures, notes- lists of.....
    'notes': music21 note/chords
    'evaluated_measure' : used in Cadenza
    'beat position' : note offset in quarter note beats 
'''
def extract_info_from_part(part):
    offsets=[] #offset in beats
    eval_measures=[] #same form as cadenza
    notes=[] #music21 note or chord
    cur_measure_offset=0

    for measure in part.getElementsByClass(music21.stream.Measure):
        
        if measure.timeSignature != None:
            cur_ts = measure.timeSignature
            beatVal = 2 **(2-math.log(cur_ts.denominator)/math.log(2))
            num_beats = cur_ts.numerator
            measure_len = beatVal * num_beats

        
        
        for note in list(measure.recurse().notes):
            offsets.append(cur_measure_offset + note.offset)
            eval_measures.append(measure.number + note.offset/4)
            notes.append(note)
        cur_measure_offset += measure_len
    
    return offsets, eval_measures, notes

In [ ]:
b = music21.converter.parse(path_to_mid)
b.show('text')


parts = b.getElementsByClass(music21.stream.Part)

for i in range(len(parts)):
    part_info= extract_info_from_part(parts[i])
    keep_times = [float(time) for time in part_info[1]]
    instr_df = df_dct[mid_instrument_lst[i]]
    #append last time 
    keep_times.append(instr_df['eval_measure'].iloc[-1])
    locations = instr_df[instr_df['eval_measure'].isin(keep_times)].index
    new_df = instr_df.loc[locations].reset_index(drop=True)
    print(part_info[2])
    print(new_df)
    df_dct[mid_instrument_lst[i]]=new_df

## Audio Stretching functions

In [ ]:
def pitch_shift_stretch1(audio_samples_full, start_sample, 
                         end_sample, prev_phase, sr, 
                         shift_constant, 
                         desired_frame_num, 
                         first_note, fft_size, hop_size, window, fft_height):

    
    '''
    We want audio from start of current frame to fft_size past ending point to calculate phases.
    Will only use last frame to calculate phase advance - will not save modulus
    
      note start 
       |----
         ----
          ---- 
           ---- note end
            ----|
             ---|-
              --|--
               -|---
                |----
    '''
    
    audio_samples = audio_samples_full[start_sample:end_sample+fft_size*2] 
    #** fft_size *2 gives some buffer audio for ptich shifting (want at least fft_size)
    #print("shape of audio_samples is ",audio_samples.shape)
    
    '''
    Part I: Perform Pitch shift on all audio 
    '''
    #print("fft_size ", fft_size)
    #print("hop size", hop_size)
    N_note=len(audio_samples)
    total_time = N_note/sr_ensemble
    
    #original timepoints
    x = np.linspace(0, total_time, num=N_note, endpoint=True)

    #interpolation function 
    sample_interpolation = interp1d(x, audio_samples, kind='linear')

    #New timepoints
    xshift = np.linspace(0, total_time, num=int(N_note/shift_constant), endpoint=True) 
    #apply interpolation function to new timepoints
    yshift = sample_interpolation(xshift)

    '''
    Part II: Time stretch
    
    We want new audio to occupy desired_frame_num amount of frames 
    Generate  desired_frames+1 timepoints from [start_sample to end_sample]
    For each timepoint except first, phase differential is difference between phase of current timepoint fft 
                and phase of current timepoint - (hop size?) fft 
                ** test what happens when this is previous index fft differential 
    
    
    If we are at beginning, initial phase is preserved as first phase difference
    Else, we are plugging in the last calculated phase difference of previous note
    '''
    
    #timpoints  
    N_shifted = int((end_sample-start_sample)/shift_constant) #want start of first note to start of next_note 
    timepoints = np.linspace(0, N_shifted, num=desired_frame_num+1, endpoint=True).round(0).astype(int)

    #iterate through creating frames - making 1 extra 
    mod = np.zeros((int(fft_size/2)+1, desired_frame_num+1))
    diff_phase = np.zeros(mod.shape)

    for i in range(desired_frame_num+1): 

        #Get position from timepoints 
        sample_index = timepoints[i]
        
        #Create fft starting from sample index 
        chunk1start=sample_index
        chunk1end=chunk1start+fft_size
        chunk1 = yshift[chunk1start:chunk1end] *window
        fft1= fft(chunk1)
        
        #Mod
        mod[:,i]=np.abs(fft1[0:fft_height ])

        #Phase 
        
        if i==0:
            
            #if it's the first note, keep its current phase 
            if first_note == True:
                frame_phase = np.angle(fft1)
                diff_phase[:,0] = frame_phase[0:fft_height ]
            #otherwise sub in last phase advance 
            else:
                diff_phase[:,0:1] = prev_phase
        else:
            #phase differential is calculated based on fft starting at cur_position - hop_size 
            chunk2start = sample_index-hop_size
                
            chunk2end = chunk2start+fft_size
            #If we're going sharp, first value will be less than 0
            if chunk2start <0:
                chunk2=np.zeros(fft_size)
                chunk2[0-chunk2start:]=yshift[0:chunk2end]
                chunk2=chunk2*window
            #normal case 
            else:
                chunk2 = yshift[chunk2start : chunk2end]*window
            fft2 = fft(chunk2) 
            
            #take difference 
            frame_phase = np.angle(fft1)[0:fft_height ]-np.angle(fft2)[0:fft_height ] 
            diff_phase[:,i]=frame_phase

    #Remove last element, saving the phase advance
    
    joining_phase_advance = diff_phase[:, -1:]
    
    
    
    diff_phase = diff_phase [:, 0:-1]
    mod=mod[:, 0:-1]
    
    return mod, diff_phase, joining_phase_advance
    




In [ ]:
def vocode_to_new_frames(df, part_samples, new_end_time):
    #new end time and new end frame should be same for everyone (if not some can be cut )

     #constants
    new_end_frame = seconds_to_stft_frames(new_end_time)
    window=scipy.signal.windows.hann(fft_size)
    fft_height = int(fft_size/2+1) #height of fft (# bins)


    #fillable arrays
    vocoded_mod= np.zeros((fft_height, new_end_frame))
    vocoded_diff_phase= np.zeros((fft_height, new_end_frame))

    start_og_sample = int(df.iloc[0]['Seconds']*sr_ensemble)#Audio frame in original playing begins
    start_new_frame = df.iloc[0]['New STFT frames']

    num_measures = len(df)
    for i in range(0, num_measures-1):

        end_og_sample = int(df.iloc[i+1]['Seconds']*sr_ensemble)
        end_new_frame = df.iloc[i+1]['New STFT frames']

        #print(start_og_sample, "start og sample")
        #print(end_og_sample, "end og sample")


        pitch_shift = df.iloc[i]['tune (abs)']
        #print("************\n"+df.iloc[i]['measure'])
        #print("shifting to pitch ", pitch_shift)
        #Calculate nessesary time stretch
        if end_og_sample < start_og_sample:
            print("END OG FRAME IS LESS THAN START OG FRAME")
            print(start_og_sample)
            print(end_og_sample)

        new_frame_dif = end_new_frame - start_new_frame


        '''
        Changes

        Hypothesis from test 2: just STFT process resutls in 1 more? frame than expected
        Resutl: 
        '''        


        #if we are on first chord, preserve initial phase 
        if i==0:


            mod, diff_phase, end_phase = pitch_shift_stretch1(audio_samples_full=part_samples,
                                                              start_sample=start_og_sample, end_sample=end_og_sample, 
                                                              prev_phase=None, sr=sr_ensemble, shift_constant=pitch_shift, 
                                                              desired_frame_num = new_frame_dif, first_note=True, 
                                                              fft_size=fft_size, hop_size=hop_size, 
                                                              window=window, fft_height=fft_height)

        else:


            mod, diff_phase, end_phase = pitch_shift_stretch1(audio_samples_full=part_samples,
                                                              start_sample=start_og_sample, end_sample=end_og_sample, 
                                                              prev_phase=end_phase, sr=sr_ensemble, shift_constant=pitch_shift, 
                                                              desired_frame_num = new_frame_dif, first_note=False, 
                                                              fft_size=fft_size, hop_size=hop_size, 
                                                              window=window, fft_height=fft_height)


        correct_len = end_new_frame-start_new_frame
        #print("length should be ", correct_len)
        #print("new chunk shape", mod.shape[1])


        #Put vocoded chunk in final STFT

        vocoded_mod[:,start_new_frame:end_new_frame]=mod
        vocoded_diff_phase[:,start_new_frame:end_new_frame]=diff_phase
        
        #increment samples

        start_og_sample = end_og_sample
        start_new_frame = end_new_frame



    
    return vocoded_mod, vocoded_diff_phase


In [ ]:

def create_final_parts(ensemble_audio_lst,df_dct,instrument_lst, end_pad ): #end pad is in ms
    adj_part_lst = []
    new_end_time = df_dct['oboe_1'].iloc[len(df_dct['oboe_1'])-1]['new_times']+end_pad
    for i in range(0, len(instrument_lst)):
        print("***************************************************************************************************")
        print("Instrument: ",instrument_lst[i] )
        #get correct stft and dataframe to describe instrument
        part_samples=ensemble_audio_lst[i]
        df = df_dct[instrument_lst[i]]
        #vocode part so it fits the new timing scheme
        vocoded_mod, vocoded_diff_phase = vocode_to_new_frames(df, part_samples, new_end_time)
        cum_resampled_phase = np.cumsum(vocoded_diff_phase, axis=1)

        #combine modulus and argument to make final stft
        cum_stft = vocoded_mod*np.exp(1j*cum_resampled_phase) 

        #add to list
        adj_part_lst.append(cum_stft)
    return adj_part_lst    


In [ ]:
def sum_parts(adj_part_lst):
        #sum parts 
    sum_vocoded_stft= np.zeros((int(fft_size/2+1), adj_part_lst[0].shape[1]), dtype='complex')
    for i in range(0, len(instrument_lst)):
        #add waves
        sum_vocoded_stft += adj_part_lst[i]*nonvar_mix[i]
        
    #convert to audio sphere
    adjusted_by_measure = librosa.istft(sum_vocoded_stft,  hop_length=hop_size)
    return adjusted_by_measure

In [ ]:
def gen_mix_parts(adj_part_lst, nonvar_mix, folder_path = None):
    
    if folder_path==None:
        print("Must supply path of folder to write parts. Create first")
        return 0
    print("writing to "+folder_path)    
    #list of parts   
    part_samples_lst = []
    
    for i in range(0, len(instrument_lst)):
        #convert into sample space
        samples_float = librosa.istft(adj_part_lst[i]*nonvar_mix[i],hop_length=hop_size)
        samples_int = (2**16*samples_float).astype(np.int16)
        part_samples_lst.append(samples_int)
        print("writing part for ", instrument_lst[i])
        write(folder_path+"/"+instrument_lst[i]+".wav", sr_ensemble, samples_int)
        

    return part_samples_lst

# Look at Dataframe

# Create df with all eval_measure and STFT frames



In [ ]:
df_STFT_dct={}

for v in instrument_lst:
    df_STFT_dct[v]= df_dct[v][['eval_measure', 'STFT frames']]
    df_STFT_dct[v][v]=df_STFT_dct[v]['STFT frames']
    df_STFT_dct[v]=df_STFT_dct[v][['eval_measure', v]]

In [ ]:
comb_df = df_STFT_dct['oboe_1'].set_index('eval_measure')

for v in instrument_lst[1:]:
    comb_df = comb_df.join(
    df_STFT_dct[v].set_index('eval_measure'), 
    #on = 'eval_measure', 
    how='outer')

In [ ]:
comb_df = comb_df.reset_index()
comb_df

### Extrapolate missing values for mean calculation

- Want tempo in interpolated sections to accelerate from tempo_old to tempo_new. 
- Method 1: Avg tempo
    - Extrapolate forward from prev section and backward from next section. Average times. 
- Method 2: Weighted avg tempo
    - Closer to forward implies tempo closer to forward, closer to backward implies tempo closer to backward. Constants calculated based from measure


In [ ]:
#part measures is df_dct[v]['eval_measure']
#max_dist is maximum gap between adjacent notes (in measures)
'''
Aim: everything under 2 measures is interpolated linearly, otherwise tempo extrapolated 
'''
def find_blocks(part_measures, max_dist = 2):
    block_lst = []
    cur_block = []
    for i in range(len(part_measures)-1):

        cur_measure = part_measures[i]
        cur_block.append(i)
        #check distance from next block
        next_measure = part_measures[i+1]

        if next_measure - cur_measure > max_dist : 
            #single notes do not constitute block
            if len(cur_block)>1:
                block_lst.append(cur_block)
            cur_block = []
    if len(cur_block)>0:
        cur_block.append(len(part_measures)-1) #add last element
        block_lst.append(cur_block)
    return block_lst   

In [ ]:
def make_extrapolate_df(nan_df, df_dct, all_measures, method = "avg"):
    extrap_df = nan_df.copy()

    for v in instrument_lst:
        x = np.array(df_dct[v]['eval_measure'])
        print(x)
        y = np.array(df_dct[v]['STFT frames']) 
        blocks = find_blocks(x)
        
        print("BLOCKS\n", blocks)
        #store extrapolations
        forward_extrapolation_lst = []
        backward_extrapolation_lst =[]
        for i in range(len(blocks)):
            print("\n******************************\nOn block", i, "\n*******")
            block = blocks[i]
            '''
            1. Interpolate within blocks to get locations of local rests
            '''
            #fit function
            
            print("fitting: ", x[block], "to", y[block])
            f = interp1d(x[block], y[block] , fill_value = 'extrapolate')
            #figure out combined index locations 
            start_measure = x[block[0]]
            end_measure = x[block[-1]]
            print("starting on measure", start_measure, "ending on measure", end_measure)
            start_comb_idx = all_measures.index(start_measure)
            end_comb_idx = all_measures.index(end_measure)
            print('In full version starts on index', start_comb_idx, "and ends on index",  end_comb_idx)
            #perform interpolation within block 
            y1 = f(all_measures[start_comb_idx: end_comb_idx+1]) #including ending index for interpolation
            print("After interpolation:", all_measures[start_comb_idx: end_comb_idx+1],
                 "maps to ", y1)

            #assign
            extrap_df[v].loc[start_comb_idx: end_comb_idx] = y1
            print("currently, dataframe is \n", extrap_df[v])
            
            '''
            2. Perform forward extrapolation 
            '''
            print("\n&&&&\nExtrapolation")
            #start extrapolation at last value of current block
            forward_extrap_start = x[blocks[i][-1]]
            #end extrapolation on first value of next block or end of piece
            if i < len(blocks)-1:
                forward_extrap_end = x[blocks[i+1][0]]
            else:
                forward_extrap_end = all_measures[-1]
            #extrapolate vals
            
            forward_start_idx = all_measures.index(forward_extrap_start)
            forward_end_idx = all_measures.index(forward_extrap_end)
            #measures slice
            forward_x = all_measures[forward_start_idx: forward_end_idx+1 ] #include end
            forward_y = f(forward_x)
            print("In forward extrapolation: mapped", forward_x, "to", forward_y)
            #make start 0 to get differences 
            forward_y = forward_y - forward_y[0]
            
            #add x y pairs to dictionary
            forward_extrapolation_lst.append({'startval':forward_start_idx, 
                                              'endval':forward_end_idx,
                                              'y':forward_y})
            '''
            2. Perform backward extrapolation 
            '''
            #start at beginning of piece or end of last block
            if i > 0:
                backward_extrap_start = x[blocks[i-1][-1]]
            else:
                backward_extrap_start =all_measures[0]
            #end on first beat of this block
            backward_extrap_end =    x[blocks[i][0]]
            
            backward_start_idx = all_measures.index(backward_extrap_start)
            backward_end_idx = all_measures.index(backward_extrap_end)
            
            
            backward_x = all_measures[backward_start_idx: backward_end_idx+1] #include end
            backward_y = f(backward_x)
            print("In backward extrapolation: mapped", backward_x, "to", backward_y)
            backward_y = backward_y - backward_y[0]
            
            backward_extrapolation_lst.append({'startval':backward_start_idx,
                                               'endval':backward_end_idx , 
                                               'y':backward_y})
 
        print("\n\n@@@@@@@@@@@@@@@\n Part 3: filling in long NAN values")

        '''
        Decide final extrapolated values
        '''
        #Fill beginning and ending values
        beginning = backward_extrapolation_lst[0]  
        print('beginning dct', beginning)
        # TODO: this should move backward from last existing value
        extrap_df[v].loc[beginning['startval']: beginning['endval']] = (
            extrap_df[v].loc[beginning['endval']]- beginning['y'][-1])+ beginning['y']
        
        print(extrap_df[v].loc[beginning['endval']]- beginning['y'][-1])
        print(beginning['y'])
        backward_extrapolation_lst.pop(0)
        
        ending = forward_extrapolation_lst[-1] 
        #for ending values, add differences on last existing value
        extrap_df[v].loc[ending['startval']: ending['endval']] = ending['y'] +extrap_df[v].loc[ending['startval']]
        forward_extrapolation_lst.pop(-1)      
        
        print("dataframe\n", extrap_df[v])
        
        print("\n\n$$$$$$$$$$\nExtrapolating long rests and moving forward\n")
        
        
        print('backward extrap info\n', backward_extrapolation_lst[0:3])
        print('forward extrap info\n', forward_extrapolation_lst[0:3])
        #Rest of long rests:
        for i in range(len(backward_extrapolation_lst)):
            print("SHOULD BE SAME")
            print("starts", backward_extrapolation_lst[i]['startval'], 
                 forward_extrapolation_lst[i]['startval'])
            print("ends", backward_extrapolation_lst[i]['endval'], 
                 forward_extrapolation_lst[i]['endval'])
            
            forward_y = forward_extrapolation_lst[i]['y']
            backward_y = backward_extrapolation_lst[i]['y']
            start_idx = forward_extrapolation_lst[i]['startval']
            end_idx = forward_extrapolation_lst[i]['endval']
            start_frame = extrap_df[v].loc[start_idx]
            end_frame = extrap_df[v].loc[end_idx]
            print("start frame", start_frame)
            print("end_frame", end_frame)
            '''
            Option 1: average:  
            '''
            print("forward_y", forward_y )
            print("backward_y",backward_y)
            y_extrap = (np.array(forward_y) + np.array(backward_y)) /2
            #add to last value
            y_extrap = y_extrap + start_frame
            
            
            '''
            Perform assignment of nan values and shifting of values after
            '''
       
            #assign extrapolation to NAN values
            print("y_extrap\n", y_extrap)
            print("putting values at ", extrap_df[v].loc[start_idx: end_idx])
            extrap_df[v].loc[start_idx: end_idx] = y_extrap
            print("df\n", extrap_df[v])
            #shift forward previous values
            shift = extrap_df[v].loc[end_idx]-end_frame
            print("next section should shift by ", shift)
            if i< len(backward_extrapolation_lst) - 1: 
                next_start_idx = forward_extrapolation_lst[i+1]['startval']
            else: 
                next_start_idx = len(all_measures)
                
            print("inserting into\n", extrap_df[v].loc[end_idx: next_start_idx])
            print("end idx", end_idx)
            print("next start idx", next_start_idx)
            extrap_df[v].loc[end_idx+1: next_start_idx] = extrap_df[v].loc[end_idx+1: next_start_idx]+shift

    return extrap_df

In [ ]:
all_measures = list(comb_df['eval_measure'])

In [ ]:
filled_comb_df = make_extrapolate_df(comb_df, df_dct, all_measures, method = "avg")
filled_comb_df

# Other utility functions

In [ ]:
def assign_new_stft(df_dct, optimized_df, instrument_lst):
    df_dct1 = df_dct.copy()
    for v in instrument_lst:

        frames_df=  optimized_df[['eval_measure']].copy()
        frames_df['New STFT frames']=optimized_df[v].astype(int) #was hardcoded oboe_1
        frames_df["new_times"] = stft_frames_to_seconds(frames_df['New STFT frames'])
        frames_df= frames_df.set_index('eval_measure')

        #join
        df_dct1[v]=df_dct[v].set_index('eval_measure').join(frames_df)
    return df_dct1

In [ ]:
def plot_tempos(x0, x_gd, del_m_series, start=0, stop=None, ylow=None, yhigh=None, show_rates=False):
    for p in range(n_p):

        d = np.diff(x0[:,p])


        e = np.diff(x_gd[:,p])


        plt.figure()
        plt.title(instrument_lst[p])
        plt.plot((d/del_m_series)[start:stop], label='old tempo')
        plt.plot((e/del_m_series)[start:stop], label = 'new ')
        if show_rates == True:
            print("Rate values", (e/del_m_series)[start:stop])
        plt.ylim([ylow, yhigh])
        plt.legend()
        plt.show()

# Loss, loss derivative, and loss hessian

## $L = L_e+ L_s + L_a + L_r$

# $L_g = \sum_{i=1}^{n_p}\sum_{j=1}^{n_b} (t_{i,j}-t^g_{i,j} )^2$

$\frac{\partial L_a}{\partial t_{i,j}} =  2(t_{i,j}-t^a_{i,j} )$

$\frac{\partial ^2 L_a}{\partial t_{i,j}^2} =  2$

where $t^a_{i,j}$ the timepoint of a close-to-perfect version


$L_r = \sum_{i=1}^{n_p}\sum_{j=1}^{n_b} (\frac{t_{i,j+1}-t_{i,j}}{m_{j+1}-m_{j}} - R_0 )^2$

where $R_0$ is average desired rate

In [ ]:
def make_der2_sq():
    der2_sq = np.ones_like(x0)* (a*(n_p-1)**2+ d*2)#ensemble and absolute loss
    
    #first range: I(b<n_b-1)
    i_del_m  = inv_del_m_series[:-1,:]
    #print('first range\n', 2*c*i_del_m * (i_del_m_p1 * del_t_p1 - i_del_m*del_t))
    der2_sq[ 0: n_b-2, : ] += 2*c*i_del_m **2

    #second range: I(1 < b < n_b)

    i_del_m = inv_del_m_series[1:, :]
    i_del_m_m1 = inv_del_m_series[:-1, :]

    der2_sq[1:-1, :] += 2*c*(-1*i_del_m - i_del_m_m1)**2
    
    #print('sec range\n', 2*c*(-1*i_del_m - i_del_m_m1) * (
     #   i_del_m*del_t - i_del_m_m1*del_t_m1))    
    #third range: b > 1

    i_del_m_m1 = inv_del_m_series[1:, :]

    der2_sq[2:, :] += 2*c*(i_del_m_m1)**2
    
    return der2_sq

In [ ]:
def make_der2_p1():
    
    #print('initial series', inv_del_m_series)
    der2_p1 = np.zeros_like(x0)
    

    #first range: I(b<n_b-1)

    i_del_m_p1 = inv_del_m_series[1:,:]
    i_del_m  = inv_del_m_series[:-1,:]
    #print('first range\n', 2*c*i_del_m * (i_del_m_p1 * del_t_p1 - i_del_m*del_t))
    der2_p1[ 0: n_b-2, : ] += 2*c*(-1*i_del_m_p1 - i_del_m) * i_del_m

    #second range: I(1 < b < n_b)
    i_del_m = inv_del_m_series[1:, :]
    i_del_m_m1 = inv_del_m_series[:-1, :]
    der2_p1[1:-1, :] += 2*c*i_del_m*(-1*i_del_m - i_del_m_m1)

    #third range: b > 1 -- no values

    return der2_p1


In [ ]:
def make_der2_m1():
    
    der2_m1 = np.zeros_like(x0)
    
    #first range: I(b<n_b-1) -- no values

    #second range: I(1 < b < n_b)
    i_del_m = inv_del_m_series[1:, :]
    i_del_m_m1 = inv_del_m_series[:-1, :]

    der2_m1[1:-1, :] += 2*c* i_del_m_m1*(-1*i_del_m - i_del_m_m1) 
    
    #print('sec range\n', 2*c*(-1*i_del_m - i_del_m_m1) * (
     #   i_del_m*del_t - i_del_m_m1*del_t_m1))    
    #third range: b > 1

    i_del_m_m1 = inv_del_m_series[1:, :]
    i_del_m_m2 = inv_del_m_series[:-1, :]

    der2_m1[2:, :] += 2*c*(-1*i_del_m_m1 - i_del_m_m2)*i_del_m_m1
    
    #print('third range\n', 2*c*(i_del_m_m1) * (
     #   i_del_m_m1*del_t_m1 - i_del_m_m2*del_t_m2))     
    return der2_m1


In [ ]:
def make_der2_p2():
    

    der2_p2 = np.zeros_like(x0)
    

    #first range: I(b<n_b-1)

    i_del_m_p1 = inv_del_m_series[1:,:]
    i_del_m  = inv_del_m_series[:-1,:]
    der2_p2[ 0: n_b-2, : ] += 2*c*i_del_m * i_del_m_p1

    #second range: I(1 < b < n_b) -- no values

    #third range: b > 1  -- no values

    return der2_p2


In [ ]:
def make_der2_m2():
    

    der2_m2 = np.zeros_like(x0)
    

    #first range: I(b<n_b-1)  -- no values

    #second range: I(1 < b < n_b) -- no values
    
    #third range: b > 1

    i_del_m_m1 = inv_del_m_series[1:, :]
    i_del_m_m2 = inv_del_m_series[:-1, :]

    der2_m2[2:, :] += 2*c*(i_del_m_m1) * (i_del_m_m2)
    
    #print('third range\n', 2*c*(i_del_m_m1) * (
     #   i_del_m_m1*del_t_m1 - i_del_m_m2*del_t_m2))     
    return der2_m2


In [ ]:
def loss_fun(x):
    
    #print(x)
    #calculate ensemble loss
    x = x.reshape(n_b, n_p)
    
    loss = np.zeros_like(x)
    
    '''
    Ensemble Loss
    '''
    ensemble_loss = np.ones_like(x)
    for p in range(n_p):
        #print("\n\n*******\n", p)
        instr_slice = x[:,other_voices_dct[p]] #initialize
        #print(instr_slice)
        cur_val = x[:, p:p+1] 
        #print(instr_slice)
        instr_sum_arr = a * np.sum((cur_val - instr_slice)**2, axis=1)  
        
        #instr_sum_arr = 2 * a * np.sum((cur_val - instr_slice)**2, axis=1) 
        #instr_sum_arr = 2 * a * np.sum((x-cur_val)**2, axis=1) 
        #print(instr_sum_arr)
        
        ensemble_loss[:,p]= instr_sum_arr
     
    #print("ensemble loss\n", loss)
    loss += ensemble_loss
    del_t_series =  np.diff(x, axis=0)
    
    
    '''
    Stretch Loss
    '''

    del_t = del_t_series[1:, :]
    del_t_m1 = del_t_series[:-1, :]

    i_del_m = inv_del_m_series[1:, :]
    i_del_m_m1 = inv_del_m_series[:-1, :]

    loss[1:-1, :] += c*(i_del_m*del_t - i_del_m_m1*del_t_m1)**2
    #print("stretch loss I(1 < b < n_b) \n", loss)    
    
    #print("stretch loss I(b>1) \n", loss)  
    
    '''
    "Absolute" Loss
    '''
    
    abs_loss = d*(x-abs_times_arr)**2
    
    loss += abs_loss
    
    print(np.sum(loss))
    return np.sum(loss)
    #return loss


In [ ]:
def loss_derivative(x):
    

    #calculate ensemble loss
    x = x.reshape(n_b, n_p)
    der = np.zeros_like(x)
    
    '''
    Ensemble Loss derivative
    '''
    
    ensemble_der = np.ones_like(x)
    for p in range(n_p):
        #print("\n\n*******\n", p)
        instr_slice = x[:,other_voices_dct[p]] #initialize
        #print(instr_slice)
        cur_val = x[:, p:p+1] 
        #print(instr_slice)
        #instr_sum_arr = 2*a*np.sum(cur_val - instr_slice, axis=1)  
        instr_sum_arr = 4*a* np.sum(cur_val-instr_slice , axis=1) 
        #print(instr_sum_arr)
        ensemble_der[:,p]= instr_sum_arr
     
    der += ensemble_der
    #print('ensemble grad\n', der)
    
    '''
    Stretch Loss Derivative
    '''
    del_t_series =  np.diff(x, axis=0)
    
    #print('del_t_serie\n', del_t_series)
    #print('inv_del_m_series', inv_del_m_series)
    #first range: I(b<n_b-1)
    del_t_p1 = del_t_series[1:,:]
    del_t = del_t_series[:-1,:]
    i_del_m_p1 = inv_del_m_series[1:,:]
    i_del_m  = inv_del_m_series[:-1,:]
    #print('first range\n', 2*c*i_del_m * (i_del_m_p1 * del_t_p1 - i_del_m*del_t))
    der[ 0: n_b-2, : ] += 2*c*i_del_m * (i_del_m_p1 * del_t_p1 - i_del_m*del_t)

    #second range: I(1 < b < n_b)
    

    del_t = del_t_series[1:, :]
    del_t_m1 = del_t_series[:-1, :]

    i_del_m = inv_del_m_series[1:, :]
    i_del_m_m1 = inv_del_m_series[:-1, :]

    der[1:-1, :] += 2*c*(-1*i_del_m - i_del_m_m1) * (
        i_del_m*del_t - i_del_m_m1*del_t_m1)
    
    #print('sec range\n', 2*c*(-1*i_del_m - i_del_m_m1) * (
     #   i_del_m*del_t - i_del_m_m1*del_t_m1))    
    #third range: b > 1
    
    del_t_m1 = del_t_series[1:, :]
    del_t_m2 = del_t_series[:-1, :]

    i_del_m_m1 = inv_del_m_series[1:, :]
    i_del_m_m2 = inv_del_m_series[:-1, :]

    der[2:, :] += 2*c*(i_del_m_m1) * (
        i_del_m_m1*del_t_m1 - i_del_m_m2*del_t_m2)
    
    #print('third range\n', 2*c*(i_del_m_m1) * (
     #   i_del_m_m1*del_t_m1 - i_del_m_m2*del_t_m2))     
        
    '''
    'Absolute' Loss derivative
    '''
    
    der += d*2*(x-abs_times_arr)
   
    return der.flatten()


In [ ]:
def loss_hessp(x, in_vec):
    
    #print('size of in_vec is ', in_vec.shape)
    in_vec = in_vec.reshape(n_b, n_p) #do this so indices match
    x = x.reshape(n_b, n_p)
    #print('x is ', x)
    Hp = np.zeros_like(x) #we will flatten this later
    #print("Hp", Hp)
    #print('element', Hp[0][0])

    
    for p in range(n_p): #which part
        for b in range(n_b): #which beat
            
            Hp[b][p] = 0
            
            #catch cross terms
            for not_p in other_voices_dct[p]: 
                Hp[b][p]+= der2_part_const* in_vec[b][not_p] #b is cur position, not_p gives block
                
                
            #Always include diagonal term
            Hp[b][p] += der2_sq[b][p]*in_vec[b][p] 
 
            #Range-dependent conditionals for stretch terms

            if b < n_b-2:
                Hp[b][p] += der2_m2[b+2][p]*in_vec[b+2][p]
                
            if b < n_b-1:
                Hp[b][p] += der2_m1[b+1][p]*in_vec[b+1][p]
                
            if b > 0:
                Hp[b][p] += der2_p1[b-1][p]*in_vec[b-1][p]
                
            if b > 1: 
                Hp[b][p] += der2_p2[b-2][p]*in_vec[b-2][p]
   
    #print('Final', Hp)  
    return Hp.flatten()    
        

## First pass: Just ensemble loss

# Alt First pass: metronomic

In [ ]:
new_timing_scheme = (filled_comb_df['eval_measure']-1)*700

In [ ]:
stft

In [ ]:
metro_time_df = filled_comb_df.copy()
for p in instrument_lst:
    metro_time_df[p]=new_timing_scheme


In [ ]:
metro_time_df[instrument_lst].to_numpy()

## New loss function adds stretch loss to absolute loss

In [ ]:
x0 = filled_comb_df[instrument_lst].to_numpy()
a =.0000005#.000001#0.000001
c = .9999999#.999999
d=.0000005
n_p = x0.shape[1]
n_b = x0.shape[0]
abs_times_arr = metro_time_df[instrument_lst].to_numpy()
#abs_times_arr = ensemble_res ['x'].reshape(n_b, n_p)
del_t_arr = np.diff(x0, axis=0)
inv_del_m_series = 1/del_t_arr
other_voices_dct = {0:[1,2,3,4,5,6,7], 
                    1:[0,2,3,4,5,6,7], 
                    2:[0,1,3,4,5,6,7],
                    3:[0,1,2,4,5,6,7],
                    4:[0,1,2,3,5,6,7],
                    5:[0,1,2,3,4,6,7],
                    6:[0,1,2,3,4,5,7],
                    7:[0,1,2,3,4,5,6]}


#Newton's method second derivatives
der2_part_const = -1*a*(n_p-1)


der2_p2 = make_der2_p2()
der2_p1 = make_der2_p1()
der2_sq = make_der2_sq()
der2_m1= make_der2_m1()
der2_m2= make_der2_m2()

In [ ]:
a+c+d

In [ ]:
import time

In [ ]:
start = time.time()
final_res = minimize(loss_fun, x0, method='Newton-CG',
               jac=loss_derivative, hessp=loss_hessp,
               options={  'disp': True})

end = time.time()
print('seconds elapsed', end - start)

In [ ]:
final_df = pd.DataFrame(final_res['x'].reshape(n_b, n_p),columns = instrument_lst, )

In [ ]:
final_df

In [ ]:
final_df=final_df+20

In [ ]:
final_df['eval_measure']= filled_comb_df['eval_measure']

In [ ]:
final_df.to_csv("grid_stretch.csv", index=False)

In [ ]:
final_stretch = assign_new_stft(df_dct, final_df, instrument_lst)
stretched_parts = create_final_parts(ensemble_audio_lst,final_stretch,instrument_lst, 2)


In [ ]:
stretched_parts[0].shape

In [ ]:
stretched_full_audio=sum_parts(stretched_parts)
ipd.Audio(stretched_full_audio, rate=sr_ensemble) 

In [ ]:
#gen_mix_parts(stretched_parts, nonvar_mix, folder_path = "./parts_audio/grid_stretch")

In [ ]:
write("whole_audio/FINALPARAMensemble_stretch.99999ensemble.000001metro.000009.wav", sr_ensemble, stretched_full_audio)

# Visualize and find error distribution

In [ ]:
final_time_arr = final_res['x'].reshape(n_b, n_p)
final_time_arr

In [ ]:
avg_times = np.array([np.sum(final_res['x'].reshape(n_b, n_p), axis=1)/num_voices]).T

In [ ]:
avg_times

In [ ]:
all_difs = final_res['x'].reshape(n_b, n_p) - avg_times

In [ ]:
from scipy.stats import iqr

In [ ]:
plt.title("3 part loss - Error distribution")
plt.hist(all_difs.flatten(), bins=100)
plt.xlabel("Difference from mean times (in frames)")
plt.ylabel("Number of timepoints")
plt.show()
print("Inter quartile range is", iqr(all_difs.flatten()))
print("Maximum deviation", np.max(np.abs(all_difs)))